In [1]:
from imgaug import augmenters as iaa
from keras import applications, optimizers
from keras.layers import Dropout, Flatten, Dense, Input
from keras.models import Sequential, Model
from keras.preprocessing.image import ImageDataGenerator
from PIL import Image
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import StratifiedShuffleSplit
import cv2
import keras
import os, csv
import shutil
import imgaug as ia
import numpy as np
import pandas as pd

Using TensorFlow backend.


In [2]:
# dimensions the images.
img_width, img_height = 512, 384

train_data_dir = '../dataset/train'
test_data_dir = '../dataset/test'
validation_data_dir = '../dataset/validation'
original_data_dir = '../dataset-resized'

batch_size = 16
epochs = 20

In [3]:
def buildModel():
    # build the VGG16 network
    base_model = applications.VGG16(weights='imagenet', 
                                    include_top=False,
                                    input_tensor=Input(shape=(img_width, img_height, 3)))

    for layer in base_model.layers:
        layer.trainable = False
        
    top_model = base_model.output
    top_model = Flatten(name="Flatten")(top_model)
    top_model = Dense(512, activation='relu')(top_model)
    top_model = Dense(256, activation='relu')(top_model)
    top_model = Dense(6, activation='softmax')(top_model)
    
    model = Model(inputs=base_model.input, outputs=top_model)

#     model.summary()
    
    model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
    
    return model

In [5]:
def generateData(batch_size):
    train_datagen = ImageDataGenerator()

    validation_datagen = ImageDataGenerator()

    train_generator = train_datagen.flow_from_directory(
        train_data_dir,
        target_size=(img_width, img_height),
        batch_size=batch_size,
        shuffle=True,
        class_mode='categorical')

    validation_generator = validation_datagen.flow_from_directory(
        validation_data_dir,
        target_size=(img_width, img_height),
        batch_size=batch_size,
        class_mode='categorical')
    
    return [train_generator,validation_generator]

In [6]:
def fineTuneModel(model, train_generator, validation_generator, epochs, batch_size):
    training_samples = 0
    validation_samples = 0
    
    for path, dirs, files in os.walk(train_data_dir):
        for filename in files:
            training_samples += 1 

    for path, dirs, files in os.walk(validation_data_dir):
        for filename in files:
            validation_samples += 1 
        
    model.fit_generator(
        train_generator,
        steps_per_epoch=training_samples // batch_size,
        epochs=epochs,
        validation_data=validation_generator,
        validation_steps=validation_samples // batch_size)
    
    return model

In [10]:
def getMetrics(model, batch_size):
    
    test_samples = 0
    
    for path, dirs, files in os.walk(test_data_dir):
        for filename in files:
            test_samples += 1 
    
    test_datagen = ImageDataGenerator(rescale=1. /255)

    test_generator = test_datagen.flow_from_directory(
        test_data_dir,
        target_size=(img_width,img_height),
        shuffle=False,
        batch_size=1,
        class_mode='categorical')
    
    predicted_results = model.predict_generator(test_generator, 
                                                steps = test_samples)
    predicted_results = np.argmax(predicted_results, axis=1)
    targets = ['cardboard', 'glass', 'metal', 'paper', 'plastic', 'trash']
    
    # confusion matrix
    print('CONFUSION MATRIX:')
    print(confusion_matrix(test_generator.classes, predicted_results))
    
    # classification report
    print('CLASSIFICATION REPORT:')
    print(classification_report(test_generator.classes, predicted_results, target_names=targets))

In [8]:
def runExperiments():
    batch_sizes = [8,16,32]
    epochs = [10,20,30]
    config = 1
    
    for batch_size in batch_sizes:
        for epoch in epochs:
            
            print("*************** Test ", config, " Batch size: ", batch_size, " Epochs: ", epoch, "***************")           
            keras.backend.clear_session()
            
            model = buildModel()
            train_generator,test_generator = generateData(batch_size)
            trained_model = fineTuneModel(model,train_generator, test_generator, epoch, batch_size)
            metrics = getMetrics(trained_model, batch_size)
            
            config += 1

In [11]:
runExperiments()

*************** Test  1  Batch size:  8  Epochs:  10 ***************
Found 3572 images belonging to 6 classes.
Found 224 images belonging to 6 classes.
Epoch 1/10
446/446 [==============================] - 125s 281ms/step - loss: 12.6348 - acc: 0.2152 - val_loss: 13.4557 - val_acc: 0.1652
Epoch 2/10
446/446 [==============================] - 124s 278ms/step - loss: 12.6307 - acc: 0.2164 - val_loss: 13.4557 - val_acc: 0.1652
Epoch 3/10
446/446 [==============================] - 125s 279ms/step - loss: 12.6261 - acc: 0.2166 - val_loss: 13.4557 - val_acc: 0.1652
Epoch 4/10
446/446 [==============================] - 125s 279ms/step - loss: 12.6487 - acc: 0.2152 - val_loss: 13.4557 - val_acc: 0.1652
Epoch 5/10
446/446 [==============================] - 125s 280ms/step - loss: 12.6397 - acc: 0.2158 - val_loss: 13.4557 - val_acc: 0.1652
Epoch 6/10
446/446 [==============================] - 125s 280ms/step - loss: 12.6216 - acc: 0.2169 - val_loss: 13.4557 - val_acc: 0.1652
Epoch 7/10
446/446 [

/home/martin/anaconda3/envs/martin/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


Found 3572 images belonging to 6 classes.
Found 224 images belonging to 6 classes.
Epoch 1/20
446/446 [==============================] - 127s 284ms/step - loss: 12.4924 - acc: 0.2239 - val_loss: 12.1605 - val_acc: 0.2455
Epoch 2/20
446/446 [==============================] - 125s 280ms/step - loss: 12.4906 - acc: 0.2251 - val_loss: 12.1605 - val_acc: 0.2455
Epoch 3/20
446/446 [==============================] - 125s 280ms/step - loss: 12.4951 - acc: 0.2248 - val_loss: 12.1605 - val_acc: 0.2455
Epoch 4/20
446/446 [==============================] - 125s 280ms/step - loss: 12.5177 - acc: 0.2234 - val_loss: 12.1605 - val_acc: 0.2455
Epoch 5/20
446/446 [==============================] - 125s 279ms/step - loss: 12.4816 - acc: 0.2256 - val_loss: 12.1605 - val_acc: 0.2455
Epoch 6/20
446/446 [==============================] - 125s 280ms/step - loss: 12.5268 - acc: 0.2228 - val_loss: 12.1605 - val_acc: 0.2455
Epoch 7/20
446/446 [==============================] - 125s 280ms/step - loss: 12.4951 - a

Found 3572 images belonging to 6 classes.
Found 224 images belonging to 6 classes.
Epoch 1/10
223/223 [==============================] - 110s 495ms/step - loss: 13.8684 - acc: 0.1376 - val_loss: 14.2472 - val_acc: 0.1161
Epoch 2/10
223/223 [==============================] - 107s 482ms/step - loss: 13.9407 - acc: 0.1351 - val_loss: 14.2472 - val_acc: 0.1161
Epoch 3/10
223/223 [==============================] - 107s 478ms/step - loss: 13.9453 - acc: 0.1348 - val_loss: 14.2472 - val_acc: 0.1161
Epoch 4/10
223/223 [==============================] - 109s 488ms/step - loss: 13.9678 - acc: 0.1334 - val_loss: 14.2472 - val_acc: 0.1161
Epoch 5/10
223/223 [==============================] - 108s 485ms/step - loss: 13.9272 - acc: 0.1359 - val_loss: 14.2472 - val_acc: 0.1161
Epoch 6/10
223/223 [==============================] - 108s 486ms/step - loss: 13.9723 - acc: 0.1331 - val_loss: 14.2472 - val_acc: 0.1161
Epoch 7/10
223/223 [==============================] - 106s 477ms/step - loss: 13.9272 - a

ResourceExhaustedError: 2 root error(s) found.
  (0) Resource exhausted: OOM when allocating tensor with shape[16,64,512,384] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[{{node block1_conv2/convolution}}]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.

	 [[loss/mul/_247]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.

  (1) Resource exhausted: OOM when allocating tensor with shape[16,64,512,384] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[{{node block1_conv2/convolution}}]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.

0 successful operations.
0 derived errors ignored.